In [1]:
# activate crawler env 1st!
import netCDF4 as nc
from netCDF4 import Dataset
import thredds_crawler
from thredds_crawler.crawl import Crawl

In [2]:
#The original problem was in Kyle's program where the thredds crawler got OPENDAP URLs
# and seemed to point to using ncattrs syntax as the problem
# trying to duplicate without having to run the crawler to get the URL
#
# this runs and does the right thing, once I got the all( if for) syntax right
# the file has MOORING and INST_TYPE, but not serial_number

required_atts = ('MOORING', 'INST_TYPE', 'serial_number')
# querying stellwagen is needed to check the EPIC atts
#fn='https://stellwagen.er.usgs.gov/thredds/catalog/TSdata/SANDWICH2016/10631whVp-cal.nc'
# this is the error generated:
#    failed trying to read https://stellwagen.er.usgs.gov/thredds/catalog/TSdata/SANDWICH2016/10631whVp-cal.nc
#fn='http://geoport.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data_tests/V_fv/SANDWICH2016/10631whVp-cal.nc'
# using gamone, because it's the most up-to-date of our TDS
fn='http://gamone.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data/SANDWICH2016/10631whVp-cal.nc'
# this is the error generated:
#   in SANDWICH2016/10631whVp-cal.nc did not bomb but is missing one of the 3 atts
spec_file = '/'.join(fn.rsplit('/', maxsplit=2)[1:])
try:
    with nc.Dataset(fn) as fnc:
        globatt_dict=fnc.__dict__
        #for k in required_attributes:
        try:
            if all(k in globatt_dict for k in required_atts):
                msg = 'in '+ spec_file + ' mooring is ' + globatt_dict['MOORING'] +': there is a '\
                    + globatt_dict['INST_TYPE'] +'; SN '+ globatt_dict['serial_number'] 
            else:
                msg="in "+ spec_file + " did not bomb but is missing one of the 3 atts"
            print(msg)
        except:
            msg="in " + spec_file +" it failed because one or more of the requred atts missing"
            print(msg)
except OSError:
    print('failed trying to read ' + fn)


in SANDWICH2016/10631whVp-cal.nc did not bomb but is missing one of the 3 atts


In [3]:
# now examine the problem file
badnc=nc.Dataset('http://gamone.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data/SANDWICH2016/10631whVp-cal.nc')

In [4]:
#demonstrates a global attribute with no '.' can be accessed with this syntax
badnc.DATA_TYPE

'WAVESPEC'

In [5]:
# I think the problem is the '.' that's in the attribute name, when used in this way 
# Python3 seems to interpret the .NBeams as a method that it can't find.
badnc.WavesMonCfg.NBeams

AttributeError: NetCDF: Attribute not found

In [6]:
#this demonstrates the '.' in the attribute is OK in a list
k='WavesMonCfg.NBeams'
if k in badnc.ncattrs():
    msg = k + ' found in global atts as list'
    print(msg)
#this demonstrates the '.' in the attribute is OK in a dict
if k in badnc.__dict__:
    msg = k + ' found in global atts as dict'
    print(msg)


WavesMonCfg.NBeams found in global atts as list
WavesMonCfg.NBeams found in global atts as dict


In [7]:
# thought maybe the __ or maybe trailing _ were problems, but seems to work in both dict and list
k='WavesMonCfg.Simulated_Sea_State_1__flat_wind_sea_swell_'
if k in badnc.ncattrs():
    msg = k + ' found in global atts as list, value is ' + str(globatt_dict[k])
    print(msg)
if k in badnc.__dict__:
    msg = k + ' found in global atts as dict'
    print(msg)


WavesMonCfg.Simulated_Sea_State_1__flat_wind_sea_swell_ found in global atts as list, value is 1.0
WavesMonCfg.Simulated_Sea_State_1__flat_wind_sea_swell_ found in global atts as dict


here's what I really need in the code to trap on all stellwagen expections

In [8]:
fpath="https://stellwagen.er.usgs.gov/thredds/catalog/TSdata/SANDWICH2016/catalog.xml"
required_atts=('MOORING', 'INST_TYPE', 'serial_number')
c = Crawl(fpath, select=[".*nc"])
urls = [s.get("url") for d in c.datasets for s in d.services if s.get("service").lower() == "opendap"]

In [9]:
for fn in urls:
    spec_file = '/'.join(fn.rsplit('/', maxsplit=2)[1:])
    try:
        with nc.Dataset(fn) as fnc:
            globatt_dict=fnc.__dict__
            #globatt_list=fnc.ncattrs()    # don't need to do as a list
            try:
                if all(k in globatt_dict for k in required_atts):
                    msg = 'in '+ spec_file + ' mooring is ' + globatt_dict['MOORING'] +': there is a '\
                           + globatt_dict['INST_TYPE'] +'; SN '+ globatt_dict['serial_number'] 
                    print (msg)
                else:
                    msg="in "+ spec_file + " did not bomb but is missing one of the 3 atts"
            except:
                msg="in " + spec_file +" it failed because one or more of the requred atts missing"
            print(msg)
    except OSError:
            print('failed trying to read ' + spec_file)

in SANDWICH2016/10591dw-a.nc mooring is 1059: there is a RBR Virtuouso D|wave; SN 055025
in SANDWICH2016/10591dw-a.nc mooring is 1059: there is a RBR Virtuouso D|wave; SN 055025
in SANDWICH2016/10591dwvs-cal.nc mooring is 1059: there is a RBR Virtuouso D|wave; SN 055025
in SANDWICH2016/10591dwvs-cal.nc mooring is 1059: there is a RBR Virtuouso D|wave; SN 055025
in SANDWICH2016/10601hwlb-a.nc mooring is 10601: there is a Hobo pressure logger; SN 10313354
in SANDWICH2016/10601hwlb-a.nc mooring is 10601: there is a Hobo pressure logger; SN 10313354
in SANDWICH2016/10602dw-a.nc mooring is 1060: there is a RBR Solo; SN 077669
in SANDWICH2016/10602dw-a.nc mooring is 1060: there is a RBR Solo; SN 077669
in SANDWICH2016/10612dw-a.nc mooring is 1061: there is a RBR Solo; SN 077827
in SANDWICH2016/10612dw-a.nc mooring is 1061: there is a RBR Solo; SN 077827
in SANDWICH2016/10622dw-a.nc mooring is 1062: there is a RBR Solo; SN 077828
in SANDWICH2016/10622dw-a.nc mooring is 1062: there is a RBR So